In [1]:
from mbientlab.metawear import MetaWear, libmetawear
from mbientlab.metawear.cbindings import *
from ctypes import c_void_p, POINTER, CFUNCTYPE, cast, byref, c_uint, c_ubyte, c_longlong
from threading import Event
import time
import csv

Exception ignored on calling ctypes callback function: <function GattChar._private_write_async.<locals>.completed at 0x000002C4B49F65C0>
Traceback (most recent call last):
  File "C:\Users\jelte\AppData\Local\Programs\Python\Python311\Lib\site-packages\mbientlab\warble\gattchar.py", line 49, in completed
    handler(WarbleException(bytes_to_str(msg)))
                            ^^^^^^^^^^^^^^^^^
  File "C:\Users\jelte\AppData\Local\Programs\Python\Python311\Lib\site-packages\mbientlab\warble\__init__.py", line 10, in bytes_to_str
    return value if sys.version_info[0] == 2 else value.decode('utf8')
                                                  ^^^^^^^^^^^^^^^^^^^^
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc0 in position 0: invalid start byte
Exception ignored on calling ctypes callback function: <function GattChar._private_write_async.<locals>.completed at 0x000002C4B49F6B60>
Traceback (most recent call last):
  File "C:\Users\jelte\AppData\Local\Programs\Python\Pytho

TypeError: expected CFunctionType instance, got NoneType

In [ ]:
# Initialize device and connect
device = MetaWear('D4:06:BD:85:8F:23')
device.connect()
e = Event()

In [3]:
# Callback for data download progress
def progress_update(context, entries_left, total_entries):
    print("Progress: {}/{}".format(total_entries - entries_left, total_entries))
    if entries_left == 0:
        e.set()

# Define function type for progress update callback
ProgressCallback = CFUNCTYPE(None, c_void_p, c_uint, c_uint)

# Instantiate the callback
progress_callback = ProgressCallback(progress_update)

# Placeholder function for unknown and unhandled entries
def placeholder_callback(context, entry_id, timestamp, data, length):
    print("Unhandled or unknown data entry received")

# Define function type for placeholder callbacks
PlaceholderCallback = CFUNCTYPE(None, c_void_p, c_ubyte, c_longlong, POINTER(c_ubyte), c_ubyte)

# Instantiate placeholder callbacks
received_unknown_entry = PlaceholderCallback(placeholder_callback)
received_unhandled_entry = PlaceholderCallback(placeholder_callback)

# Callback to handle downloaded data
data_entries = []

def data_handler(context, data):
    if data.contents.type_id == DataTypeId.CARTESIAN_FLOAT:
        entry = cast(data.contents.value, POINTER(CartesianFloat)).contents
        epoch = data.contents.epoch
        data_entries.append([epoch, entry.x, entry.y, entry.z])

DataCallback = CFUNCTYPE(None, c_void_p, POINTER(Data))
data_callback = DataCallback(data_handler)

# Function to setup and start logger for a sensor
def setup_sensor_logging(sensor_fn, data_signal):
    # Callback to handle logger creation
    def logger_created(context, new_logger):
        if new_logger:
            libmetawear.mbl_mw_logger_subscribe(cast(new_logger, c_void_p), None, data_callback)
            sensor_fn(device.board)

    LoggerCreatedCallback = CFUNCTYPE(None, c_void_p, c_void_p)
    logger_callback = LoggerCreatedCallback(logger_created)
    libmetawear.mbl_mw_datasignal_log(data_signal, None, logger_callback)

# Get data signals
acc_signal = libmetawear.mbl_mw_acc_get_acceleration_data_signal(device.board)
gyro_signal = libmetawear.mbl_mw_gyro_bmi160_get_rotation_data_signal(device.board)

# Setup logging
setup_sensor_logging(libmetawear.mbl_mw_acc_enable_acceleration_sampling, acc_signal)
setup_sensor_logging(libmetawear.mbl_mw_gyro_bmi160_enable_rotation_sampling, gyro_signal)

# Start logging and sensors
libmetawear.mbl_mw_logging_start(device.board, 0)
libmetawear.mbl_mw_acc_start(device.board)
libmetawear.mbl_mw_gyro_bmi160_start(device.board)

# Log data for 10 seconds
time.sleep(10)

# Stop sensors and logging
libmetawear.mbl_mw_acc_stop(device.board)
libmetawear.mbl_mw_gyro_bmi160_stop(device.board)
libmetawear.mbl_mw_logging_stop(device.board)

# Download data
print("Downloading data...")
download_handler = LogDownloadHandler(received_progress_update=progress_callback,
                                      received_unknown_entry=received_unknown_entry,
                                      received_unhandled_entry=received_unhandled_entry)
libmetawear.mbl_mw_logging_download(device.board, 0, byref(download_handler))
e.wait()

# Write to CSV
with open('sensor_data.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['Timestamp', 'Gyroscope X', 'Gyroscope Y', 'Gyroscope Z', 'Accelerometer X', 'Accelerometer Y', 'Accelerometer Z'])
    for entry in data_entries:
        writer.writerow(entry)

print("Data logged to sensor_data.csv")


Exception ignored on calling ctypes callback function: <function GattChar._private_write_async.<locals>.completed at 0x000002C4B49F68E0>
Traceback (most recent call last):
  File "C:\Users\jelte\AppData\Local\Programs\Python\Python311\Lib\site-packages\mbientlab\warble\gattchar.py", line 49, in completed
    handler(WarbleException(bytes_to_str(msg)))
                            ^^^^^^^^^^^^^^^^^
  File "C:\Users\jelte\AppData\Local\Programs\Python\Python311\Lib\site-packages\mbientlab\warble\__init__.py", line 10, in bytes_to_str
    return value if sys.version_info[0] == 2 else value.decode('utf8')
                                                  ^^^^^^^^^^^^^^^^^^^^
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc0 in position 0: invalid start byte
Exception ignored on calling ctypes callback function: <function GattChar._private_write_async.<locals>.completed at 0x000002C4B40202C0>
Traceback (most recent call last):
  File "C:\Users\jelte\AppData\Local\Programs\Python\Pytho

TypeError: incompatible types, CFunctionType instance instead of CFunctionType instance

In [1]:
device.on_disconnect = lambda status: print ("we are disconnected!")
device.disconnect()

NameError: name 'device' is not defined